In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/book_recommendation')
os.listdir('./data')

Mounted at /content/drive


['2000users_120interactions_100000booksample_bert_embeddings_fantasy.json.gz',
 'goodreads_interactions_2k_users_fantasy_paranormal.json.gz',
 'goodreads_interactions_20k_users_book_ids.csv',
 'goodreads_books_fantasy_paranormal.json.gz',
 'smaller_books_100000.json.gz',
 'goodreads_interactions_fantasy_paranormal.json.gz',
 'goodreads_interactions_2k_users_book_ids.csv',
 'goodreads_interactions_2k_users.json.gz']

In [2]:
# Verified necessary libraries
import numpy as np
import pandas as pd
import json
import gzip

In [ ]:
import pandas as pd
import ijson
import requests
import os
import gzip
import json
import re
import os
import sys
import numpy as np
import pandas as pd

from transformers import BertModel, BertTokenizer
import torch

from collections import defaultdict, Counter

ModuleNotFoundError: No module named 'ijson'

In [3]:
DIR = './data'
print(os.listdir(DIR))
USE_SMALLSET = True

['2000users_120interactions_100000booksample_bert_embeddings_fantasy.json.gz', 'goodreads_interactions_2k_users_fantasy_paranormal.json.gz', 'goodreads_interactions_20k_users_book_ids.csv', 'goodreads_books_fantasy_paranormal.json.gz', 'smaller_books_100000.json.gz', 'goodreads_interactions_fantasy_paranormal.json.gz', 'goodreads_interactions_2k_users_book_ids.csv', 'goodreads_interactions_2k_users.json.gz']


In [4]:
print(os.curdir)
df = pd.read_csv(os.path.join(DIR, "goodreads_interactions_2k_users_book_ids.csv"))
print(df.head())
print(df.count())

.
    book_id
0    102969
1  25293197
2  18080377
3  13083783
4  25903297
book_id    81867
dtype: int64


In [ ]:
def create_smaller_books(file_name, book_id_df, count):
    data = []
    # convert book ids from dataframe to set
    book_ids = book_id_df["book_id"].values.tolist()
    book_ids = [str(x) for x in book_ids]
    # open gzip and read json lines
    outfile_name = os.path.join(DIR,'smaller_books_'+str(count)+'.json.gz')
    with gzip.open(file_name, 'rt') as fin, gzip.open(outfile_name, 'wt') as fout:
        linecount = 0
        for line in fin:
            # convert json line to dictionary
            book = json.loads(line)
            fout.write(line)
            linecount += 1
            if linecount == count:
                break

    return data

In [ ]:
smaller_book_count = 100000

In [ ]:
create_smaller_books(os.path.join(DIR, "goodreads_books_fantasy_paranormal.json.gz"), df, smaller_book_count)

[]

In [ ]:
def load_books(file_name, book_id_df, count = -1, max_iter = -1):
    data = []
    # convert book ids from dataframe to set
    book_ids = book_id_df["book_id"].values.tolist()
    book_ids = [str(x) for x in book_ids]
    # open gzip and read json lines
    with gzip.open(file_name, 'rt') as f:
        iter = 0
        for line in f:
            iter += 1
            # convert json line to dictionary
            book = json.loads(line)
            # check if book id is in the set
            id = (book['book_id'])
            if (book['book_id']) in book_ids:
                count -= 1
                data.append(book)
            if count == 0:
                break
            if(iter == max_iter):
                break
        if(iter % 1000 == 0):
            print(iter)

    return data

In [ ]:
if(USE_SMALLSET):
    infile = 'smaller_books_'+str(smaller_book_count)+'.json.gz'
else:
    infile = 'goodreads_books.json.gz'

books = load_books(os.path.join(DIR, infile), df, -1)
print("Number of books: ", len(books))
print(books[0])

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(books)
df.to_json(os.path.join(DIR, "goodreads_interactions_20k_users_books.json.gz"), orient="records", compression="gzip")

Generate embeddings for obtained books

In [ ]:
import torch
x = torch.rand(5, 3)
print(x)


tensor([[0.4474, 0.4698, 0.3909],
        [0.5437, 0.8783, 0.3055],
        [0.5091, 0.3499, 0.4088],
        [0.6373, 0.3073, 0.6229],
        [0.6841, 0.4462, 0.5326]])


In [ ]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to generate BERT embeddings
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Apply the function to the 'description' column
df['bert_embeddings'] = df['description'].apply(lambda x: get_bert_embeddings(x) if pd.notnull(x) else None)

print(df['bert_embeddings'].head())

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

c:\Users\agoel\.conda\envs\cs229Proj\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\agoel\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


0    [[-0.34174502, -0.50928676, 0.1366887, -0.6276...
1    [[0.0080082845, -0.085280105, 0.23509698, -0.1...
2    [[-0.0092276335, -0.22185859, -0.34482706, 0.0...
3    [[-0.11977858, 0.041211575, -0.19592418, -0.13...
4    [[-0.31632385, -0.1351991, 0.3966409, -0.03750...
Name: bert_embeddings, dtype: object


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight and fast
df['bert_embeddings'] = df['description'].apply(lambda x: model.encode(x) if pd.notnull(x) else None)